In [15]:
#files to open: GRU_sequence_weights.h5, folder Search/search, output, train.csv

from google.colab import files

uploaded = files.upload()


Saving train.csv to train.csv


In [0]:
import os
import shutil 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


def deltas_in_search(file_name = 'search'):
    file_search = open(file_name, 'r')

    count = 0
    for line in file_search:
        count +=1
        if count == 5:
            break
        continue

    deltas = {}

    for line in file_search:
        if line.find('Searching') != -1:
            J0 = int(line.split()[1])
            deltas[J0] = []
            E0 = float(line.split()[-1])
            continue

        if line.find('No') != -1 or \
           len(line.split()) == 0 or \
           int(line.split()[0]) != int(J0) - 1:
            continue

        try:
            E = float(line.split()[-1])
            delta = round(E - E0, 3)
            deltas[J0].append(delta)
        except ValueError:
            break
            
    return deltas


In [4]:
import csv
import random
random.seed(227)

file_csv = open('output_predictions', 'w', newline='')

filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6])
for i in range(int(len(data)/2), len(data)):
    filewriter.writerow(data[i])

#files.download('train.csv')

NameError: ignored

In [0]:
file_csv.close()
files.download('test.csv')


In [5]:
from keras.models import Model
from keras.layers import Dense, Input, GRU


def model_GRU(input_shape, Ty):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(32, return_sequences=False)(i)
    #X = LSTM(16, return_sequences=Flase)(i)
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model

Using TensorFlow backend.


In [0]:
# LSTM model

In [0]:
Tx, Ty =  6, 1

LSTM_model = model((Tx,1),Ty)
print(LSTM_model.summary())
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('sources/LSTM_sequence_weights.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6, 1)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                1152      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________
None


# GRU model

In [0]:
Tx, Ty =  6, 1

GRU_model = model_GRU((Tx,1),Ty)
#print(LSTM_model.summary())
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#GRU_model.load_weights('sources/GRU_sequence_weights.h5')
GRU_model.load_weights('GRU_sequence_weights.h5')

# Calculate deltas on base on ready search file

In [0]:
seria_name, quan_num = 'J1J-1', [1, 1, 0]
peak_file = 'expfile_st'

In [19]:
deltas = deltas_in_search()
print(deltas)

{1: [-0.504, -0.499, -0.489, -0.484, -0.462, -0.448, -0.44, -0.419, -0.391, -0.385, -0.38, -0.333, -0.327, -0.289, -0.262, -0.244, -0.233, -0.224, -0.193, -0.147, -0.117, -0.088, -0.066, -0.056, -0.042, -0.011, -0.001, 0.003, 0.047, 0.075, 0.091, 0.131, 0.136, 0.201, 0.216, 0.256, 0.285, 0.327, 0.332, 0.346, 0.388, 0.398, 0.407, 0.413, 0.424, 0.434, 0.472], 2: [-0.496, -0.486, -0.481, -0.472, -0.464, -0.453, -0.438, -0.435, -0.426, -0.421, -0.417, -0.411, -0.405, -0.391, -0.38, -0.361, -0.343, -0.332, -0.323, -0.314, -0.309, -0.289, -0.283, -0.268, -0.248, -0.243, -0.227, -0.212, -0.201, -0.194, -0.182, -0.171, -0.164, -0.159, -0.125, -0.119, -0.114, -0.108, -0.047, -0.036, -0.031, -0.021, -0.005, 0.034, 0.048, 0.067, 0.072, 0.076, 0.086, 0.103, 0.116, 0.124, 0.141, 0.145, 0.152, 0.161, 0.166, 0.18, 0.197, 0.21, 0.247, 0.251, 0.257, 0.275, 0.281, 0.285, 0.295, 0.312, 0.324, 0.335, 0.353, 0.358, 0.363, 0.375, 0.392, 0.397, 0.402, 0.419, 0.432, 0.439, 0.444, 0.448, 0.454, 0.478, 0.484, 0

# Make sequences and PREDICTION 

In [20]:
numb_seq = 1
for k,v in deltas.items():
    numb_seq *= len(deltas[k])
print(numb_seq/1000000,  'millions')

172672.6232 millions


In [0]:
max_numb = 10000

In [0]:
data_train = pd.read_csv('train.csv')
X = data_train.drop(['Y'], axis='columns').values

ss = StandardScaler()
X = ss.fit_transform(X)


In [23]:
keys = list(deltas.keys())
ini_key = keys[0]
sequence = []
#data = []
counter = 0
exit = False

def tree_2(k, seq_ini, counter=counter):
    for v in deltas[k]:
        sequence = seq_ini.copy()
        sequence.append(v)
        if counter % 500000 == 0:
                print('!',counter)
                #a=input()
        if len(sequence) == len(deltas.items()):
            counter += 1
            sequence_np = np.array([sequence])
            sequence_np = ss.transform(sequence_np)
            sequence_np = np.expand_dims(sequence_np, axis=2)
            prediction = GRU_model.predict(sequence_np)

            if prediction > 0.6: 
                print(prediction, sequence)

        else:
            counter = tree_2(k+1, sequence,counter=counter)
        
    return counter
        
tree_2(ini_key,sequence, counter=counter)

! 0
! 0
! 0
! 0
! 0
! 0
[[0.70833385]] [-0.504, -0.496, -0.501, -0.5, -0.486, -0.502]
[[0.6555739]] [-0.504, -0.496, -0.501, -0.5, -0.486, -0.495]
[[0.7138525]] [-0.504, -0.496, -0.501, -0.5, -0.481, -0.502]
[[0.6619129]] [-0.504, -0.496, -0.501, -0.5, -0.481, -0.495]
[[0.7194118]] [-0.504, -0.496, -0.501, -0.5, -0.476, -0.502]
[[0.66831505]] [-0.504, -0.496, -0.501, -0.5, -0.476, -0.495]
[[0.72584677]] [-0.504, -0.496, -0.501, -0.5, -0.47, -0.502]
[[0.6760545]] [-0.504, -0.496, -0.501, -0.5, -0.47, -0.495]
[[0.7296161]] [-0.504, -0.496, -0.501, -0.5, -0.464, -0.502]
[[0.6832176]] [-0.504, -0.496, -0.501, -0.5, -0.464, -0.495]
[[0.7321668]] [-0.504, -0.496, -0.501, -0.5, -0.46, -0.502]
[[0.6870963]] [-0.504, -0.496, -0.501, -0.5, -0.46, -0.495]
[[0.76262784]] [-0.504, -0.496, -0.501, -0.5, -0.433, -0.502]
[[0.7228775]] [-0.504, -0.496, -0.501, -0.5, -0.433, -0.495]
[[0.6395359]] [-0.504, -0.496, -0.501, -0.5, -0.433, -0.483]
[[0.77102464]] [-0.504, -0.496, -0.501, -0.5, -0.425, -0.502]

KeyboardInterrupt: ignored